In [2]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

### Matrix tiling algorithm with Transpositon/Register/Rectangular


In [3]:
render_code("matrix_mul/rect_blockmm_trans.c", show=["//START","//END"])

// matrix_mul/rect_blockmm_trans.c:73-96 (24 lines)
//START
void blockmm_transpose(double **a, double **b, double **c, uint64_t M, uint64_t N, uint64_t K)
{
  int i,j,k, ii, jj, kk;
  double result=0;
  for(i = 0; i < M; i+=tile_size_y)
  {
    for(j = 0; j < K; j+=tile_size_y)
    {
      for(k = 0; k < N; k+=tile_size_x)
      {
          for(ii = i; ii < i+tile_size_y; ii++)
              for(jj = j; jj < j+tile_size_y; jj++)
              {
                      result = 0;
                      for(kk = k; kk < k+tile_size_x ; kk++)
                          result += a[ii][kk]*b[jj][kk];
                      c[ii][jj] += result;
              }
      }
    }
  }  
}
//END

In [7]:
display_df_mono(render_csv("matrix_mul/blockmm_transpose.csv"))

,index,size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,0,512,8,1099865947,256656369,0.233352,0.185587,0.047632,0.009279,4173628,449812388
1,1,1024,8,8752656667,2027981203,0.231699,0.182578,0.370264,0.008217,29409424,3579302787
2,2,2048,8,70399112328,16435832965,0.233466,0.179150,2.944487,0.001985,57154865,28791258099
3,3,4096,8,563048261047,133588085917,0.237259,0.179115,23.927594,0.002511,578090929,230264417563
4,4,2048,8,70408494537,16474898796,0.233990,0.179050,2.949823,0.003063,88187750,28795120136
5,5,2048,16,64863961959,14040845180,0.216466,0.179152,2.515449,0.027740,750900783,27069370660
6,6,2048,32,62449092729,15013932980,0.240419,0.179006,2.687589,0.041441,1093575993,26388884006
7,7,2048,64,61287300182,13738422737,0.224164,0.179182,2.461678,0.023440,611204904,26075713629
8,8,2048,128,60766823678,17192655415,0.282928,0.178999,3.077470,0.017444,452605027,25946188195
9,9,2048,256,60495059818,17659377139,0.291914,0.179154,3.163753,0.024576,635958937,25877202822


In [3]:
! cd matrix_mul; make rect_blockmm_trans; echo "size,tile_size_x,tile_size_y,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > rect_blockmm_trans.csv
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 8 8 >> ./matrix_mul/rect_blockmm_trans.csv 
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 8 16 >> ./matrix_mul/rect_blockmm_trans.csv 
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 16 8 >> ./matrix_mul/rect_blockmm_trans.csv
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 16 16 >> ./matrix_mul/rect_blockmm_trans.csv
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 32 8 >> ./matrix_mul/rect_blockmm_trans.csv 
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 32 16 >> ./matrix_mul/rect_blockmm_trans.csv 
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 64 8 >> ./matrix_mul/rect_blockmm_trans.csv
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 128 8 >> ./matrix_mul/rect_blockmm_trans.csv
! taskset -c 8 ./matrix_mul/rect_blockmm_trans 2048 256 8 >> ./matrix_mul/rect_blockmm_trans.csv
display_df_mono(render_csv("matrix_mul/rect_blockmm_trans.csv"))

make: 'rect_blockmm_trans' is up to date.
10521102336.000000,10521102336.000000,10521102336.000000,10521102336.000000,10521102336.000000,10521102336.000000,10521102336.000000,10521102336.000000,10521102336.000000,

,index,size,tile_size_x,tile_size_y,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses
0,0,2048,8,8,60695555037,11657275893,0.192061,0.192798,2.247494,0.007132,112916297,15831872431
1,1,2048,8,16,59756704568,12542666241,0.209896,0.192797,2.418194,0.013378,205684269,15374836543
2,2,2048,16,8,49689254796,11478181153,0.230999,0.192801,2.213007,0.023224,283706291,12216228471
3,3,2048,16,16,49214520874,9689317253,0.196879,0.193166,1.871642,0.033904,406359244,11985703960
4,4,2048,32,8,44182865583,9834559204,0.222588,0.192802,1.896127,0.027058,281603064,10407210747
5,5,2048,32,16,43946744211,9780966192,0.222564,0.192821,1.885977,0.065379,672906507,10292358338
6,6,2048,64,8,41432551019,10366196265,0.250194,0.192806,1.998660,0.029767,282898837,9503765017
7,7,2048,128,8,40060051136,11983443147,0.299137,0.192807,2.310493,0.017104,154839243,9053028767
8,8,2048,256,8,39377390928,14401358397,0.365727,0.193070,2.780473,0.031576,278783055,8828987177


## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [ ]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

Now, let's take a look of what's happening!

In [ ]:
! cd prefetch; make clean; make
# ! echo "Without prefetch -- the baseline"; ssh htseng@celebi "lscpu | grep Model; cd courses/CS203/demo/memory/prefetch/; ./transpose"
! echo "Without prefetch -- the baseline"
! lscpu | grep Model
! ./prefetch/transpose
! echo "With prefetch"
! ./prefetch/transpose_prefetch

Let's try a different machine now.

In [ ]:
! ssh htseng@xerneas "cd /nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@xerneas  "/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@xerneas  "/nfshome/htseng/courses/CSE142/demo/software_optimizations_memory/prefetch/transpose_prefetch"

In [ ]:
! ssh htseng@blissey "cd /nfshome/htseng/courses/CSE142/demo/memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@blissey  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@blissey  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose_prefetch"

In [ ]:
! ssh htseng@eevee "cd /nfshome/htseng/courses/CSE142/demo/memory/; make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; ssh htseng@eevee  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose"
! echo "With prefetch";  ssh htseng@eevee  "/nfshome/htseng/courses/CSE142/demo/memory/prefetch/transpose_prefetch"


-- It doesn't work always!

In [ ]:
render_code("matrix_mul/blockmm_interchange.c", show=["//START","//END"])

In [ ]:
! cd matrix_mul; rm -f blockmm_interchange; make blockmm_interchange; echo "size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm_interchange.csv
! ./matrix_mul/blockmm_interchange 2048 8 >> ./matrix_mul/blockmm_interchange.csv 
! ./matrix_mul/blockmm_interchange 2048 16 >> ./matrix_mul/blockmm_interchange.csv 
! ./matrix_mul/blockmm_interchange 2048 32 >> ./matrix_mul/blockmm_interchange.csv 
! ./matrix_mul/blockmm_interchange 2048 64 >> ./matrix_mul/blockmm_interchange.csv
! ./matrix_mul/blockmm_interchange 2048 128 >> ./matrix_mul/blockmm_interchange.csv
! ./matrix_mul/blockmm_interchange 2048 256 >> ./matrix_mul/blockmm_interchange.csv
! cd matrix_mul; echo "size,tile_size,IC,Cycles,CPI,CT_ns,ET_s,DL1_miss_rate,DL1_misses,DL1_accesses" > blockmm.csv
! ./matrix_mul/blockmm 2048 16 >> ./matrix_mul/blockmm.csv 
! ./matrix_mul/blockmm 2048 32 >> ./matrix_mul/blockmm.csv 
! ./matrix_mul/blockmm 2048 64 >> ./matrix_mul/blockmm.csv
! ./matrix_mul/blockmm 2048 128 >> ./matrix_mul/blockmm.csv 
! ./matrix_mul/blockmm 2048 256 >> ./matrix_mul/blockmm.csv 
! ./matrix_mul/blockmm 2048 512 >> ./matrix_mul/blockmm.csv 
display_df_mono(render_csv("matrix_mul/blockmm.csv"))
display_df_mono(render_csv("matrix_mul/blockmm_interchange.csv"))
